In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta

offer = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Codestates AI Bootcamp/Project/Project3/db/departure_mod.csv")
prediction = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Codestates AI Bootcamp/Project/Project3/db/prediction.csv")

In [3]:
offer.head()

,number,seg_id_1,code,total_dur,dur_1,iatacode_1,iatacode_2,carriercode_1,air_number_1,at_1,transfer,seg_id_1.1,dur_2,iatacode_1.1,iatacode_2.1,carriercode_2,air_number_2,at_2,bookable_seats,currency,total
0,1,175,73H,PT8H30M,PT2H25M,LGW,MAD,UX,1014,2022-03-01,Y,175,PT2H,MAD,ORY,UX,1025.0,2022-03-01,9,EUR,83.51
1,2,94,320,PT1H20M,PT1H20M,LHR,NaN,AF,1381,2022-03-01,N,94,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84
2,3,95,320,PT1H20M,PT1H20M,LHR,NaN,AF,1581,2022-03-01,N,95,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84
3,4,97,320,PT1H20M,PT1H20M,LHR,NaN,AF,1781,2022-03-01,N,97,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84
4,5,98,320,PT1H20M,PT1H20M,LHR,NaN,AF,1281,2022-03-01,N,98,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84


In [4]:
prediction.head()

,id,dest,date,ontime_proba,number
0,1,PAR,2022-03-01,0.821371,1
1,2,PAR,2022-03-02,0.818650,2
2,3,PAR,2022-03-03,0.797806,3
3,4,PAR,2022-03-04,0.753250,4
4,5,PAR,2022-03-05,0.807903,5


In [5]:
prediction["dest"].value_counts()

PAR    31
MAD    31
FRA    31
ROM    31
ICN    31
Name: dest, dtype: int64

In [6]:
offer["iatacode_2.1"].value_counts()

FRA    4759
FCO    3682
MAD    3541
CDG    3324
ICN    2415
ORY     341
Name: iatacode_2.1, dtype: int64

In [7]:
def country(x):
    if x == "FRA":
        return "FRA"
    elif x == "FCO":
        return "ROM"
    elif x == "MAD":
        return "MAD"
    elif x == "CDG" or x == "ORY":
        return "PAR"
    elif x == "ICN":
        return "ICN"

In [8]:
offer["dest"] = offer["iatacode_2.1"].apply(lambda x : country(x))

In [9]:
offer["dest_2"] = offer["iatacode_1.1"].apply(lambda x : country(x))

In [10]:
offer.head()

,number,seg_id_1,code,total_dur,dur_1,iatacode_1,iatacode_2,carriercode_1,air_number_1,at_1,transfer,seg_id_1.1,dur_2,iatacode_1.1,iatacode_2.1,carriercode_2,air_number_2,at_2,bookable_seats,currency,total,dest,dest_2
0,1,175,73H,PT8H30M,PT2H25M,LGW,MAD,UX,1014,2022-03-01,Y,175,PT2H,MAD,ORY,UX,1025.0,2022-03-01,9,EUR,83.51,PAR,MAD
1,2,94,320,PT1H20M,PT1H20M,LHR,NaN,AF,1381,2022-03-01,N,94,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,None,PAR
2,3,95,320,PT1H20M,PT1H20M,LHR,NaN,AF,1581,2022-03-01,N,95,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,None,PAR
3,4,97,320,PT1H20M,PT1H20M,LHR,NaN,AF,1781,2022-03-01,N,97,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,None,PAR
4,5,98,320,PT1H20M,PT1H20M,LHR,NaN,AF,1281,2022-03-01,N,98,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,None,PAR


In [11]:
offer["dest"] = offer["dest"].fillna(offer["dest_2"])

In [12]:
offer = offer.drop(columns=["dest_2"], axis=1)

In [13]:
offer["arr_date"] = offer["at_2"]
offer["arr_date_2"] = offer["at_1"]

In [14]:
offer["arr_date"] = offer["arr_date"].fillna(offer["arr_date_2"])

In [15]:
offer = offer.drop(columns=["arr_date_2"], axis=1)

In [16]:
offer.head()

,number,seg_id_1,code,total_dur,dur_1,iatacode_1,iatacode_2,carriercode_1,air_number_1,at_1,transfer,seg_id_1.1,dur_2,iatacode_1.1,iatacode_2.1,carriercode_2,air_number_2,at_2,bookable_seats,currency,total,dest,arr_date
0,1,175,73H,PT8H30M,PT2H25M,LGW,MAD,UX,1014,2022-03-01,Y,175,PT2H,MAD,ORY,UX,1025.0,2022-03-01,9,EUR,83.51,PAR,2022-03-01
1,2,94,320,PT1H20M,PT1H20M,LHR,NaN,AF,1381,2022-03-01,N,94,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01
2,3,95,320,PT1H20M,PT1H20M,LHR,NaN,AF,1581,2022-03-01,N,95,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01
3,4,97,320,PT1H20M,PT1H20M,LHR,NaN,AF,1781,2022-03-01,N,97,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01
4,5,98,320,PT1H20M,PT1H20M,LHR,NaN,AF,1281,2022-03-01,N,98,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01


In [17]:
prediction = prediction.drop(columns=["id", "number"], axis=1)

In [18]:
prediction.rename(columns={"date" : "arr_date"}, inplace=True)

In [19]:
prediction.head()

,dest,arr_date,ontime_proba
0,PAR,2022-03-01,0.821371
1,PAR,2022-03-02,0.818650
2,PAR,2022-03-03,0.797806
3,PAR,2022-03-04,0.753250
4,PAR,2022-03-05,0.807903


In [20]:
offer = pd.merge(left=offer, right=prediction, how="left", on=["dest", "arr_date"])

In [21]:
offer

,number,seg_id_1,code,total_dur,dur_1,iatacode_1,iatacode_2,carriercode_1,air_number_1,at_1,transfer,seg_id_1.1,dur_2,iatacode_1.1,iatacode_2.1,carriercode_2,air_number_2,at_2,bookable_seats,currency,total,dest,arr_date,ontime_proba
0,1,175,73H,PT8H30M,PT2H25M,LGW,MAD,UX,1014,2022-03-01,Y,175,PT2H,MAD,ORY,UX,1025.0,2022-03-01,9,EUR,83.51,PAR,2022-03-01,0.821371
1,2,94,320,PT1H20M,PT1H20M,LHR,NaN,AF,1381,2022-03-01,N,94,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01,0.821371
2,3,95,320,PT1H20M,PT1H20M,LHR,NaN,AF,1581,2022-03-01,N,95,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01,0.821371
3,4,97,320,PT1H20M,PT1H20M,LHR,NaN,AF,1781,2022-03-01,N,97,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01,0.821371
4,5,98,320,PT1H20M,PT1H20M,LHR,NaN,AF,1281,2022-03-01,N,98,NaN,CDG,NaN,NaN,NaN,NaN,9,EUR,85.84,PAR,2022-03-01,0.821371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20334,20335,139,77W,PT27H45M,PT13H5M,LHR,SIN,SQ,321,2022-03-31,Y,139,PT6H25M,SIN,ICN,SQ,612.0,2022-04-02,9,EUR,1131.76,ICN,2022-04-02,NaN
20335,20336,10,388,PT29H,PT13H5M,LHR,SIN,SQ,319,2022-03-31,Y,10,PT6H25M,SIN,ICN,SQ,612.0,2022-04-02,9,EUR,1131.76,ICN,2022-04-02,NaN
20336,20337,158,788,PT22H50M,PT11H55M,LHR,NRT,JL,42,2022-03-31,Y,158,PT2H35M,HND,ICN,JL,5205.0,2022-04-01,9,EUR,2854.81,ICN,2022-04-01,NaN
20337,20338,64,788,PT24H15M,PT7H40M,LHR,YYZ,AC,853,2022-03-31,Y,64,PT13H45M,YYZ,ICN,AC,61.0,2022-03-31,9,EUR,3142.24,ICN,2022-03-31,0.786006


In [22]:
offer = offer.drop(columns=["number", "seg_id_1", "seg_id_1.1", "air_number_1", 'air_number_2', 'at_2'])

In [23]:
offer.columns

Index(['code', 'total_dur', 'dur_1', 'iatacode_1', 'iatacode_2',
       'carriercode_1', 'at_1', 'transfer', 'dur_2', 'iatacode_1.1',
       'iatacode_2.1', 'carriercode_2', 'bookable_seats', 'currency', 'total',
       'dest', 'arr_date', 'ontime_proba'],
      dtype='object')

In [24]:
offer.columns = ['code', 'total_dur', 'dur_1', "departure_1", "arrival_1", 'carriercode_1', "dep_date", 'transfer', 'dur_2',
                 "departure_2", "arrival_2", 'carriercode_2', 'bookable_seats', 'currency', 'total', 'dest', 'arr_date', 'ontime_proba']

In [25]:
offer = offer[['code', 'total_dur', 'dur_1', "departure_1", "arrival_1", 'carriercode_1', "dep_date", 'transfer', 'dur_2',
                 "departure_2", "arrival_2", 'carriercode_2', 'arr_date', 'bookable_seats', 'currency', 'total', 'dest', 'ontime_proba']]

In [26]:
offer

,code,total_dur,dur_1,departure_1,arrival_1,carriercode_1,dep_date,transfer,dur_2,departure_2,arrival_2,carriercode_2,arr_date,bookable_seats,currency,total,dest,ontime_proba
0,73H,PT8H30M,PT2H25M,LGW,MAD,UX,2022-03-01,Y,PT2H,MAD,ORY,UX,2022-03-01,9,EUR,83.51,PAR,0.821371
1,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
2,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
3,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
4,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20334,77W,PT27H45M,PT13H5M,LHR,SIN,SQ,2022-03-31,Y,PT6H25M,SIN,ICN,SQ,2022-04-02,9,EUR,1131.76,ICN,NaN
20335,388,PT29H,PT13H5M,LHR,SIN,SQ,2022-03-31,Y,PT6H25M,SIN,ICN,SQ,2022-04-02,9,EUR,1131.76,ICN,NaN
20336,788,PT22H50M,PT11H55M,LHR,NRT,JL,2022-03-31,Y,PT2H35M,HND,ICN,JL,2022-04-01,9,EUR,2854.81,ICN,NaN
20337,788,PT24H15M,PT7H40M,LHR,YYZ,AC,2022-03-31,Y,PT13H45M,YYZ,ICN,AC,2022-03-31,9,EUR,3142.24,ICN,0.786006


In [27]:
pip install isodate

In [28]:
import isodate

In [29]:
# offer["dur_2"] = offer["dur_2"].fillna("PT0H00M")

In [30]:
# offer["total_dur"] = offer["total_dur"].apply(isodate.parse_duration)
# offer["dur_1"] = offer["dur_1"].apply(isodate.parse_duration)
# offer["dur_2"] = offer["dur_2"].apply(isodate.parse_duration)

In [31]:
# offer["dep_date"] = offer["dep_date"].apply(pd.to_datetime)

In [32]:
offer

,code,total_dur,dur_1,departure_1,arrival_1,carriercode_1,dep_date,transfer,dur_2,departure_2,arrival_2,carriercode_2,arr_date,bookable_seats,currency,total,dest,ontime_proba
0,73H,PT8H30M,PT2H25M,LGW,MAD,UX,2022-03-01,Y,PT2H,MAD,ORY,UX,2022-03-01,9,EUR,83.51,PAR,0.821371
1,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
2,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
3,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
4,320,PT1H20M,PT1H20M,LHR,NaN,AF,2022-03-01,N,NaN,CDG,NaN,NaN,2022-03-01,9,EUR,85.84,PAR,0.821371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20334,77W,PT27H45M,PT13H5M,LHR,SIN,SQ,2022-03-31,Y,PT6H25M,SIN,ICN,SQ,2022-04-02,9,EUR,1131.76,ICN,NaN
20335,388,PT29H,PT13H5M,LHR,SIN,SQ,2022-03-31,Y,PT6H25M,SIN,ICN,SQ,2022-04-02,9,EUR,1131.76,ICN,NaN
20336,788,PT22H50M,PT11H55M,LHR,NRT,JL,2022-03-31,Y,PT2H35M,HND,ICN,JL,2022-04-01,9,EUR,2854.81,ICN,NaN
20337,788,PT24H15M,PT7H40M,LHR,YYZ,AC,2022-03-31,Y,PT13H45M,YYZ,ICN,AC,2022-03-31,9,EUR,3142.24,ICN,0.786006


In [33]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(offer, train_size=0.8, random_state=42)

In [34]:
train, val = train_test_split(train, train_size=0.8, random_state=42)

In [35]:
train.shape, val.shape, test.shape

((13016, 18), (3255, 18), (4068, 18))

In [46]:
target = "total"
features = "dep_date"
# features = offer.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((13016,), (13016,), (3255,), (3255,), (4068,), (4068,))

In [47]:
# 회귀 기준모델 생성
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 기준 예측값
predicted = y_train.mean()
y_pred_base = [predicted] * len(y_val)

mae = mean_absolute_error(y_val, y_pred_base)
mse = mean_squared_error(y_val, y_pred_base)
r2 = r2_score(y_val, y_pred_base)

print("기준 모델 (평균) :", predicted)
print("기준 모델 MAE :", mae)
print("기준 모델 MSE :", mse)
print("기준 모델 R2 score :", r2)

기준 모델 (평균) : 277.3684288568015
기준 모델 MAE : 177.60290923543116
기준 모델 MSE : 171663.4821246482
기준 모델 R2 score : -0.00035903759483635334


In [48]:
pip install category-encoders

In [49]:
# 그래디언트 부스팅으로 회귀 계산

from xgboost import XGBRegressor
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)

In [50]:
boosting = XGBRegressor(
    n_estimators=1000,
    objective="reg:squarederror",
    learning_rate=0.2,
    n_jobs=-1
)

eval_set = [(X_train_encoded, y_train),
            (X_val_encoded, y_val)]

boosting.fit(X_train_encoded, y_train,
             eval_set=eval_set,
             early_stopping_rounds=100)

[0]	validation_0-rmse:475.949	validation_1-rmse:473.545
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:456.959	validation_1-rmse:453.781
[2]	validation_0-rmse:444.377	validation_1-rmse:440.563
[3]	validation_0-rmse:436.126	validation_1-rmse:431.776
[4]	validation_0-rmse:430.755	validation_1-rmse:425.972
[5]	validation_0-rmse:427.278	validation_1-rmse:422.144
[6]	validation_0-rmse:425.036	validation_1-rmse:419.63
[7]	validation_0-rmse:423.581	validation_1-rmse:417.995
[8]	validation_0-rmse:422.619	validation_1-rmse:416.931
[9]	validation_0-rmse:422.022	validation_1-rmse:416.185
[10]	validation_0-rmse:421.627	validation_1-rmse:415.702
[11]	validation_0-rmse:421.381	validation_1-rmse:415.375
[12]	validation_0-rmse:421.2	validation_1-rmse:415.18
[13]	validation_0-rmse:421.094	validation_1-rmse:415.021
[14]	validation_0-rmse:421.022	validation_1-rmse:414.922

XGBRegressor(learning_rate=0.2, n_estimators=1000, n_jobs=-1,
             objective='reg:squarederror')

In [51]:
y_pred_val_xgb = boosting.predict(X_val_encoded)
y_pred_test_xgb = boosting.predict(X_test_encoded)

print("XGB val mae :", mean_absolute_error(y_val, y_pred_val_xgb))
print("XGB val mse :", mean_squared_error(y_val, y_pred_val_xgb))
print("XGB val R2 score :", r2_score(y_val, y_pred_val_xgb))

XGB val mae : 176.10361300054456
XGB val mse : 172020.9652644318
XGB val R2 score : -0.0024422499661820307


In [64]:
X_test_encoded

,dep_date
4202,16
2879,9
3728,24
15556,5
20011,24
...,...
9352,26
16119,10
14962,6
6418,2


In [63]:
y_pred_test_xgb

array([275.58832, 275.05212, 268.15784, ..., 253.71144, 284.68436,
       303.69962], dtype=float32)

In [52]:
print("XGB MAE_test :", mean_absolute_error(y_test, y_pred_test_xgb))
print("XGB MSE_test :", mean_squared_error(y_test, y_pred_test_xgb))
print("XGB R2 score_test :", r2_score(y_test, y_pred_test_xgb))

XGB MAE_test : 160.29734322877056
XGB MSE_test : 144082.51440453655
XGB R2 score_test : -0.0026862542508208076


In [53]:
boosting.save_model("xgb_model.model")

In [55]:
X_test_encoded

,dep_date
4202,16
2879,9
3728,24
15556,5
20011,24
...,...
9352,26
16119,10
14962,6
6418,2


In [65]:
pip install nbconvert